# Check dye input files
**Author: Jun Sasaki  Coded 2024-12-26  Updated 2025-04-17**<br>
Checking input files for dye simulation. Demonstration of a static plot using matplotlib and an interactive plot using hvplot.

In [ ]:
import os
from xfvcom import FvcomDataLoader, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import get_index_by_value
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfile = "obc_dye2020.nc"
ncfile_path = os.path.join(base_path, ncfile)
ds = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5).ds
ds

### Retrieve a list of open boundary nodes

In [ ]:
obc_nodes =ds.obc_nodes.values
print(f"obc_nodes={obc_nodes}")

### Retrieve the node index `node_idx` corresponding to a specified `node` value and create its DataArray.

In [ ]:
node = 3137
node_idx = get_index_by_value(obc_nodes, node)
print(f"node_idx={node_idx}")
da = ds['obc_dye'].isel(nobc=node_idx)

### Plot timeseries contourf
Further costomization can be made using `(fig, ax, cbar)`. 

In [ ]:
png_dir = "PNG"; os.makedirs(png_dir, exist_ok=True)
png_file = "dye_obc.png"
png_path = os.path.join(png_dir, png_file)
cfg = FvcomPlotConfig(figsize=(10,2))
plotter = FvcomPlotter(ds, cfg)
# xlim = (da.time[0].values, da.time[-1].values)
ylim = (1, 0)
# vmin, vmax = (da.min().item(), da.max().item())
vmin, vmax = (2.5, 6)
title = f"Dye at node={node}"
contourf_kwargs=dict(vmin=vmin, vmax=vmax, levels=31)
colorbar_kwargs=dict(label="Dye")
fig, ax, cbar = plotter.ts_contourf(da, ylim=ylim, title=title, contourf_kwargs=contourf_kwargs, colorbar_kwargs=colorbar_kwargs)
fig.savefig(png_path, dpi=600, bbox_inches='tight')

## Spatial 2D plot along open boundary nodes at a specified time

In [ ]:
print(ds["siglay"].dims)      # 何が出ますか?
print(ds["siglay"].shape)

In [ ]:
# ───────────────────────────────────────────────
# 0) prerequisites
# ───────────────────────────────────────────────
selected_time = np.datetime64("2020-01-01")
dye_profile   = ds["obc_dye"].sel(time=selected_time, method="nearest")
# dims: ('siglay', 'nobc')

# ───────────────────────────────────────────────
# 1) build σ-grid, 水深グリッド, 境界ノード番号グリッド
# ※ シンボル名は従来コードに合わせる
# ───────────────────────────────────────────────
siglay_1d   = ds["siglay"].values              # shape: (nsiglay,)
water_depth = ds["obc_h"].values               # shape: (nnobc,)

# ⇒ ('siglay','nobc') にブロードキャスト
siglay_grid  = np.broadcast_to(siglay_1d[:, None],
                               (siglay_1d.size, water_depth.size))
nobc_indices = np.broadcast_to(np.arange(water_depth.size),
                               (siglay_1d.size, water_depth.size))
depth        = siglay_grid * water_depth[None, :]     # 実深度（負値）

# ───────────────────────────────────────────────
# 2) plotting (元コードそのまま動く)
# ───────────────────────────────────────────────
plt.close()
yaxis_siglay = False          # ← True なら σ 座標、False なら実深度
plt.figure(figsize=(10, 3))

if yaxis_siglay:
    plt.contourf(
        nobc_indices,          # X 軸（境界ノードインデックス）
        siglay_grid,           # Y 軸（σレイヤ値・負値で上が 0）
        dye_profile.values,    # Z （濃度）
        levels=20, cmap="viridis"
    )
else:
    plt.contourf(
        nobc_indices,          # X
        depth,                 # Y: 実深度（負値）
        dye_profile.values,    # Z
        levels=20, cmap="viridis"
    )

plt.colorbar(label="Dye Concentration")
plt.xlabel("Boundary Node Index")
plt.ylabel("Depth (m)")
plt.title(f"Dye Concentration Cross-Section on {selected_time}")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
%%skip
selected_time = '2020-01-01'
selected_time = np.datetime64(selected_time)
dye_profile = ds['obc_dye'].sel(time=selected_time, method='nearest')
siglay = ds['siglay'][:, node_idx]
obc_h = ds['obc_h']
depth = siglay * obc_h  # Determine the depth at each node
depth = depth.transpose('siglay', 'nobc') 
nobc_indices = np.tile(obc_h.nobc.values, (len(siglay), 1)) 
nobc_numbers = np.tile(obc_nodes, (len(siglay),1))
siglay_grid = np.tile(siglay.values[:, None], (1, len(obc_nodes)))

In [ ]:
%%skip
plt.close()
yaxis_siglay=False
plt.figure(figsize=(10, 3))
if yaxis_siglay:
    plt.contourf(
        nobc_indices, siglay_grid, dye_profile.values, levels=20, cmap='viridis'
    )
else:
    plt.contourf(
        nobc_indices, depth, dye_profile.values, levels=20, cmap='viridis'
    )
plt.colorbar(label="Dye Concentration")
plt.xlabel("Boundary Node Index")
plt.ylabel("Depth (m)")
plt.title(f"Dye Concentration Cross-Section on {selected_time}")
plt.gca().invert_yaxis()  # 水深を正の方向に（深くなるほど下へ）
#plt.grid(True)
plt.tight_layout()
plt.show()

## Interactive timeseries plot at a specified node
Useful for checking values interactively.

In [ ]:
import holoviews as hv
import panel as pn
hv.extension('bokeh')
pn.extension()

In [ ]:
def plot_time_siglay(node):
    """
    The dye concentration at the specified node is plotted as a heatmap based on time series and sigma layers (siglay).

    Parameters:
    -----------
    node(int): Specified node number
    """
    # Retrieve data at the specified node while deleting nob.
    dye_profile = ds['obc_dye'].isel(nobc=node, drop=True)  # (time, siglay)
    
    # plot with 'time' vs 'siglay'
    heatmap = dye_profile.hvplot.quadmesh(
        x='time',
        y='siglay',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Node {node} Over Time",
        xlabel='Time',
        ylabel='Sigma Layer'
    )
    
    return heatmap

In [ ]:
node_select = pn.widgets.Select(
    name='Node Index',
    options=list(range(ds.sizes['nobc'])),  # 0〜12
    value=0
)

interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# Create a layout
layout1 = pn.Column(
    "### Dye Concentration Over Time",
    node_select,
    interactive_plot1
)
final_layout = pn.Row(layout1)
final_layout

## Interactive spatial 2D plot along open boundary nodes at the specified time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import panel as pn

# ------------------------------------------------------------
# 関数本体
# ------------------------------------------------------------
def plot_node_depth(time: pd.Timestamp) -> plt.Figure:
    """
    Plot dye concentration vs. real depth at all boundary nodes for a given time.

    Parameters
    ----------
    time : pandas.Timestamp
        Selected time.
    """
    # 1) slice dye concentration (dims: 'siglay', 'nobc')
    dye_prof = ds["obc_dye"].sel(time=time, method="nearest")

    # 2) 1-D sigma, 1-D water depth
    siglay_1d      = ds["siglay"].values           # shape: (nsiglay,)
    water_depth_1d = ds["obc_h"].values            # shape: (nnobc,)

    # 3) broadcast to 2-D grids
    depth = np.outer(siglay_1d, water_depth_1d)    # ('siglay','nobc')
    nobc_indices = np.broadcast_to(
        np.arange(water_depth_1d.size),
        depth.shape
    )

    # 4) plot
    fig, ax = plt.subplots(figsize=(6, 2))
    pcm = ax.contourf(
        nobc_indices,        # X: boundary‐node index
        depth,               # Y: real depth (negative downward)
        dye_prof.values,     # Z: concentration
        levels=20, cmap="viridis"
    )
    fig.colorbar(pcm, ax=ax, label="Dye Concentration")
    ax.set(
        xlabel="Boundary Node Index",
        ylabel="Depth (m)",
        title=f"Dye Concentration Cross-Section on {time}"
    )
    ax.invert_yaxis()
    fig.tight_layout()
    plt.close(fig)
    return fig

In [ ]:
# A widget for selecting time
time_options = [pd.to_datetime(t).strftime('%Y-%m-%d %H:%M') for t in ds['time'].values]

time_select = pn.widgets.DiscreteSlider(
    name='Time',
    options = time_options,  # list of str
    value = time_options[0]
)

# Create an interactive plot
interactive_plot2 = pn.bind(plot_node_depth, time=time_select)

# Create a layout
layout2 = pn.Column(
    "# Dye Concentration by Depth",
    time_select,
    interactive_plot2
)
final_layout = pn.Row(layout2)
final_layout